In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 124.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


## Set up model definition

In [6]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the base LLama model
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [8]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of Llama3

In [9]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('HF_TOKEN'))


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
fn_out = '/content/drive/MyDrive/w266_project/Meta-Llama-3-8B-Instruct-output-reformatted-reference.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [11]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):

  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question: "+sample}
  ]

  outputs = pipe(
      prompt,
      max_new_tokens=1024,
      temperature=0.01,
      return_full_text=False
  )
  response = outputs[0]["generated_text"]
  response = response.replace('\n','\t')
  print(response)
  with open(fn_out, 'a') as f:
      f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the patient's symptoms and medication use, the most likely cause of her gastrointestinal blood loss is gastrointestinal bleeding due to naproxen-induced gastritis and gastric ulcers. The coffee-ground vomitus is a classic sign of upper gastrointestinal bleeding, and the patient's age and osteoarthritis may have contributed to her increased risk of gastrointestinal complications.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In the context of disseminated intravascular coagulation (DIC), the blood component expected to show an increase is D-dimer. D-dimer is a degradation product of cross-linked fibrin, which is formed during the breakdown of fibrin clots. In DIC, excessive breakdown of fibrin leads to an increase in D-dimer levels, which can be used as a diagnostic marker for this condition.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Based on the patient's history and reduced heroin use, it is likely that she is being prescribed an opioid analgesic, such as morphine or oxycodone. The primary mechanism of action of these drugs is to bind to opioid receptors in the brain and spinal cord, thereby reducing the transmission of pain signals and providing analgesia.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the diagnosis of papillary thyroid carcinoma with no lymph node involvement, the appropriate surgical management for this patient would be a total thyroidectomy (removal of both lobes of the thyroid gland) with central compartment lymph node dissection (removal of lymph nodes in the central compartment of the neck). This approach provides optimal surgical staging and minimizes the risk of recurrent disease.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the patient's symptoms and laboratory results, a stone is most likely to be found in the common bile duct (CBD) or gallbladder. The presence of scleral icterus, diffuse upper abdominal tenderness, and elevated liver enzymes (alkaline phosphatase and γ-glutamyltransferase) suggest a diagnosis of cholecystitis or choledocholithiasis. Ultrasonography is likely to reveal a gallstone or CBD stone, which is causing the patient's symptoms.
Based on the symptoms, the most likely diagnosis is Condyloma acuminatum, also known as genital warts. The characteristic features of this condition include:		* Recurrent, scaly lesions on the glans penis	* Lesions appearing at the same location	* Lesions healing with slight hyperpigmentation		Genital warts are caused by the human papillomavirus (HPV) and are typically treated with topical medications, cryotherapy, or surgical excision.
